In [1]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import pandas as pd
import math
import keras
import os
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
import sklearn.preprocessing as sk
import random
from tensorflow.python.keras import backend as K
from sklearn.impute import SimpleImputer
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
GDSC_Expr = pd.read_csv("./Updated Data/GDSCv2.exprs.BDKANN.V3.tsv", sep = "\t", index_col=0, decimal = ".")

CTRP_Expr = pd.read_csv("./Updated Data/CTRP.exprs.BDKANN.V3.tsv", 
                    sep = "\t", index_col=0, decimal = ".")

#transpose so that input data is cell lines on rows and genes on columns 
GDSC_Expr = GDSC_Expr.T
CTRP_Expr = CTRP_Expr.T

GDSC_resp = pd.read_csv("./Updated Data/GDSCv2.aac.BDKANN.V3.tsv",sep="\t")

CTRP_resp = pd.read_csv("./Updated Data/CTRPv2.aac.BDKANN.V3.tsv",sep="\t")

GDSC_resp = GDSC_resp.T
CTRP_resp = CTRP_resp.T

# Match samples/indexes between expression and response
idx_GDSC = GDSC_Expr.index.intersection(GDSC_resp.index)
idx_CTRP = CTRP_Expr.index.intersection(CTRP_resp.index)
GDSC_Expr = GDSC_Expr.loc[idx_GDSC,:]
GDSC_resp = GDSC_resp.loc[idx_GDSC,:]
CTRP_Expr = CTRP_Expr.loc[idx_CTRP,:]
CTRP_resp = CTRP_resp.loc[idx_CTRP,:]

#Remove tissue type features and store in new DF 
GDSC_Expr_genes = GDSC_Expr.loc[:,GDSC_Expr.columns.str.contains('ENS')]
GDSC_tissue_type = GDSC_Expr.loc[:,~GDSC_Expr.columns.str.contains('ENS')]
CTRP_Expr_genes = CTRP_Expr.loc[:,CTRP_Expr.columns.str.contains('ENS')]
CTRP_tissue_type = CTRP_Expr.loc[:,~CTRP_Expr.columns.str.contains('ENS')]

In [9]:
GDSC_tissue_type.columns

Index(['CNS/Brain', 'Bone', 'Prostate', 'Esophagus/Stomach',
       'Bladder/Urinary Tract', 'Ovary/Fallopian Tube', 'Lymphoid', 'Kidney',
       'Thyroid', 'Soft Tissue', 'Head and Neck', 'Skin', 'Lung', 'Pleura',
       'Myeloid', 'Uterus', 'Pancreas', 'Breast', 'Cervix', 'Bowel',
       'Peripheral Nervous System', 'Liver', 'Biliary Tract', 'Other',
       'Ampulla of Vater'],
      dtype='object')

In [4]:
Mask1 = pd.read_csv("./Masks/M1New.csv", 
                    sep = ",", index_col=0, decimal = ".")

Mask2 = pd.read_csv("./Masks/M2New.csv", 
                    sep = ",", index_col=0, decimal = ".")

gene_names = pd.read_csv('./Masks/mart_export(3).txt',sep='\t')

gene_names = gene_names.drop_duplicates('NCBI gene (formerly Entrezgene) ID')

gene_dict = dict(zip(gene_names.iloc[:,1],gene_names.iloc[:,2]))

# Convert ensembl ids to entrez gene ids  and re-order to match mask 
GDSC_Expr = GDSC_Expr_genes.loc[:,GDSC_Expr_genes.columns.intersection(gene_names.iloc[:,1])]
GDSC_Expr = GDSC_Expr.rename(columns=gene_dict)
GDSC_Expr = GDSC_Expr.sort_index(axis=1)

CTRP_Expr = CTRP_Expr_genes.loc[:,CTRP_Expr_genes.columns.intersection(gene_names.iloc[:,1])]
CTRP_Expr = CTRP_Expr.rename(columns=gene_dict)
CTRP_Expr = CTRP_Expr.sort_index(axis=1)

Mask1 = Mask1.loc[CTRP_Expr.columns,:]

drugs = ['ATRA', 'Doxorubicin', 'Tamoxifen', 'Gefitinib', 'BIBW2992','masitinib', '17-AAG', 'GDC-0941', 'MK-2206', 'AZD6244', 'PLX4720']
drugIDs = ['15367', '28748', '41774', '49668', '61390', '63450', '64153', '65326', '67271','90227', '90295']
drug_dict = dict(zip(drugs,drugIDs))
GDSC_resp = GDSC_resp[GDSC_resp.columns.intersection(drugs)]
CTRP_resp = CTRP_resp[GDSC_resp.columns.intersection(drugs)]

CTRP_resp = CTRP_resp.rename(columns=drug_dict)
GDSC_resp = GDSC_resp.rename(columns=drug_dict)

imputer1 = SimpleImputer(missing_values=np.nan, strategy='mean')
GDSC_resp = imputer1.fit_transform(GDSC_resp.values)

imputer1 = SimpleImputer(missing_values=np.nan, strategy='mean')
CTRP_resp = imputer1.fit_transform(CTRP_resp.values)

GDSC_all_feats = pd.concat([GDSC_Expr,GDSC_tissue_type],axis=1)
CTRP_all_feats = pd.concat([CTRP_Expr,CTRP_tissue_type],axis=1)

Mask1 = pd.concat([Mask1, pd.DataFrame(np.ones((25, Mask1.shape[1])), columns=Mask1.columns)])

lsP = Mask1.columns.intersection(Mask2.index)

Mask1 = Mask1.loc[:,lsP]
Mask2 = Mask2.loc[lsP,:]

Mask3 = pd.read_csv("./Masks/M3New.csv", 
                    sep = ",", index_col=0, decimal = ".")
lsD = Mask2.columns.intersection(Mask3.index)
Mask2 = Mask2.loc[:,lsD]
Mask3 = Mask3.loc[lsD,['49668', '67271', '90295', '41774']]

# Mask1v2 = 1 - Mask1.values
# Mask2v2 = 1 - Mask2.values
# Mask3v2 = 1 - Mask3.values

In [5]:
# Regularization using masks for each layer
def l1_regL1(weight_matrix):
    return 0.001 * K.sum(K.abs(weight_matrix*Mask1v2))
def l1_regL2(weight_matrix):
    return 0.001 * K.sum(K.abs(weight_matrix*Mask2v2))
def l1_regL3(weight_matrix):
    return 0.001 * K.sum(K.abs(weight_matrix*Mask3v2))

In [6]:
def create_BDKANN2_model():
    """create BDKANN2 (with regularized extra edges)"""
    model = Sequential()
    model.add(Dense(Mask2.shape[0], input_dim=Mask1.shape[0], kernel_initializer='random_normal', activation='relu', 
                    kernel_regularizer=l1_regL1))
    model.add(Dense(Mask2.shape[1], activation='relu', kernel_initializer='random_normal', kernel_regularizer=l1_regL2))
    model.add(Dense(GDSC_resp.shape[1], activation='linear', kernel_initializer='random_normal', kernel_regularizer=l1_regL3))
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    return model

In [7]:
def train_model(X_train, y_train, model_fn, model_file):
    model = model_fn()
    # model callbacks 
    early_stopping = EarlyStopping(monitor='val_mse', patience=10, verbose=1, mode='auto',restore_best_weights=True)
    mc = ModelCheckpoint(filepath=model_file, monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
    
    history = model.fit(X_train, y_train, epochs=100, batch_size=16,  verbose=0, validation_split=0.2, callbacks=[early_stopping,mc])
    return model

In [8]:
def remove_connections(mask, percent=0.05):
    one_idx = np.argwhere(mask == 1)
    set_to_zero = np.random.choice(len(one_idx), math.ceil(len(one_idx)*percent),replace=False)
    mask_new = np.copy(mask)
    for i in set_to_zero:
        top_idx = one_idx[i]
        mask_new[top_idx[0],top_idx[1]] = 0
    return 1-mask_new, one_idx[set_to_zero]

In [10]:
X_train = CTRP_all_feats
X_test = GDSC_all_feats
y_train = CTRP_resp
y_test = GDSC_resp

scaler = sk.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

total_stats = []
for run in range(10):
    Mask1v2, rem_idx_1  = remove_connections(Mask1.values)
    Mask2v2, rem_idx_2 = remove_connections(Mask2.values)
    Mask3v2, rem_idx_3 = remove_connections(Mask3.values)

    layer_idxs = [rem_idx_1, rem_idx_2, rem_idx_3]

    model = train_model(X_train, y_train, create_BDKANN2_model, model_file='./Models/Recover/BDKANN+_recover_{}.h5'.format(run))

    percent_recovered = []
    for l, layer in enumerate(model.layers):
        weights = layer.get_weights()[0]
        dropped_weights = [weights[i[0],i[1]] for i in layer_idxs[l]]

        percent_recovered.append(np.sum(dropped_weights>weights.mean())/len(layer_idxs[l]))

        #print('Mean of all weights = {:.6f}, Mean of dropped weights = {:.6f}'.format(weights.mean(), np.mean(dropped_weights)))
    total_stats.append(percent_recovered)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4448)              2072768   
_________________________________________________________________
dense_1 (Dense)              (None, 867)               3857283   
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 3472      
Total params: 5,933,523
Trainable params: 5,933,523
Non-trainable params: 0
_________________________________________________________________
Restoring model weights from the end of the best epoch.
Epoch 00085: early stopping
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4448)              2072768   
_________________________________________________________________
dense_

In [11]:
recovered_connections = pd.DataFrame(total_stats)

In [14]:
recovered_connections

,0,1,2
0,0.264696,0.520833,0.0
1,0.237867,0.562500,1.0
2,0.379699,0.907738,1.0
3,0.239747,0.529762,0.0
4,0.289474,0.770833,0.0
5,0.253418,0.586310,0.0
6,0.200957,0.568452,0.0
7,0.227614,0.571429,0.0
8,0.199248,0.613095,0.0
9,0.265208,0.532738,1.0


In [13]:
recovered_connections.to_csv('Results/recovered_connections_percent_v2.csv')

In [12]:
recovered_connections.mean(), recovered_connections.std()

(0    0.255793
 1    0.616369
 2    0.300000
 dtype: float64,
 0    0.051895
 1    0.124948
 2    0.483046
 dtype: float64)